# Exploration des données pour conseiller nos clients sur l'achat de nos véhicules disponibles sur notre plateforme AutoPredict


Vous pourrez, ci-dessous, consulter nos statistiques pour savoir quels types de véhicules nous pourrions vous orienter en fonction de vos besoins.

In [297]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go

# Activation du mode notebook (important pour VS Code)
pyo.init_notebook_mode(connected=True)

# Chargement du fichier CSV
df = pd.read_csv("voiture.csv")

print(f"Dataset size before cleaning: {len(df)} rows")

# Data cleaning
# Standardize column names
df.columns = df.columns.str.lower().str.replace(" ", "_")
df.rename(columns={
    'engine_fuel_type': 'fuel_type',
    'engine_hp': 'hp',
    'engine_cylinders': 'cylinders',
    'transmission_type': 'transmission',
    'driven_wheels': 'drive',
    'number_of_doors': 'doors',
    'market_category': 'market',
    'vehicle_size': 'size',
    'vehicle_style': 'style',
    'msrp': 'price'
}, inplace=True)

# Handle missing values
df['fuel_type'] = df['fuel_type'].fillna('Non défini')
df['hp'] = df['hp'].fillna(df['hp'].mean()).round().astype('Int64')
df['cylinders'] = df['cylinders'].fillna(0)
df['doors'] = df['doors'].fillna(df['doors'].mean()).round().astype('Int64')
df['market'] = df['market'].fillna('Non défini')


# Add derived columns
df['average_mpg'] = (df['highway_mpg'] + df['city_mpg']) / 2
df['hp_range'] = pd.qcut(df['hp'], q=5, labels=['Très faible', 'Faible', 'Moyenne', 'Élevée', 'Très élevée'])
df['cylinders'] = df['cylinders'].round().astype('Int64')

# Final dataset information
print(f"Dataset size after cleaning: {len(df)} rows")
print("Number of missing values in each column:")
print(df.isnull().sum())

Dataset size before cleaning: 11914 rows
Dataset size after cleaning: 11914 rows
Number of missing values in each column:
make            0
model           0
year            0
fuel_type       0
hp              0
cylinders       0
transmission    0
drive           0
doors           0
market          0
size            0
style           0
highway_mpg     0
city_mpg        0
popularity      0
price           0
average_mpg     0
hp_range        0
dtype: int64


In [298]:
# Préparation des données
df_clean = df[['transmission', 'style']].dropna()
df_clean = df_clean[df_clean['transmission'] != 'UNKNOWN']

# Agrégation par transmission et style
style_trans = df_clean.groupby(['transmission', 'style']).size().reset_index(name='Nombre de véhicules')

# Création du graphique sunburst
fig = px.sunburst(
    style_trans,
    path=['transmission', 'style'],
    values='Nombre de véhicules',
    title="Répartition des styles de véhicule selon le type de transmission"
)

# Affichage du pourcentage et étiquettes
fig.update_traces(
    textinfo='label+percent entry',
    insidetextorientation='radial'
)

# Taille du graphique
fig.update_layout(
    font=dict(size=14),
    width=900,
    height=700
)

# Affichage dans VS Code
pyo.iplot(fig)


In [299]:
# Suppression des lignes avec valeurs manquantes
df_clean = df.dropna(subset=['hp', 'highway_mpg', 'city_mpg', 'cylinders']).copy()

# Calcul de la consommation moyenne
df_clean['Average MPG'] = (df_clean['highway_mpg'] + df_clean['city_mpg']) / 2

# Filtrage : suppression des moteurs à 0 cylindre (valeurs aberrantes)
df_clean = df_clean[df_clean['cylinders'] > 0]

# Tranches de puissance moteur
df_clean['HP Range'] = pd.qcut(df_clean['hp'], q=5, labels=[
    'Très faible', 'Faible', 'Moyenne', 'Élevée', 'Très élevée'])

# Conversion des cylindres à entier
df_clean['cylinders'] = df_clean['cylinders'].round().astype('Int64')

# Agrégation
df_filtered = df_clean.groupby(['HP Range', 'cylinders']).agg(
    avg_mpg=('Average MPG', 'mean'),
    std_mpg=('Average MPG', 'std')
).reset_index()

# Graphique interactif
fig = px.line(
    df_filtered,
    x='HP Range',
    y='avg_mpg',
    error_y='std_mpg',
    color='cylinders',
    markers=True,
    title="Consommation moyenne par puissance moteur",
    labels={
        "HP Range": "Type de puissance moteur",
        "avg_mpg": "Consommation moyenne (MPG)",
        "cylinders": "Nombre de cylindres"
    }
)

fig.update_layout(
    legend_title_text='Nb Cylindres',
    yaxis_title="Consommation moyenne (MPG)",
    xaxis_title="Type de puissance moteur"
)

fig.show()


C:\Users\jerem\AppData\Local\Temp\ipykernel_19480\1954858973.py:18: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [300]:
# Nettoyage et préparation
df_clean = df.dropna(subset=['hp', 'market', 'price']).copy()
df_clean['HP Range'] = pd.qcut(df_clean['hp'], q=5, labels=[
    'Très faible', 'Faible', 'Moyenne', 'Élevée', 'Très élevée'])
df_clean['market'] = df_clean['market'].str.split(',')
df_exploded = df_clean.explode('market')

# Calcul du prix moyen
df_price = df_exploded.groupby(['HP Range', 'market'])['price'].mean().reset_index()
df_price['price'] = df_price['price'].round(0)

# Nombre de véhicules
vehicle_counts = df_exploded.groupby(['HP Range', 'market']).size().reset_index(name='Nombre de véhicules')
df_price_annotated = df_price.merge(vehicle_counts, on=['HP Range', 'market'])

# Création du graphique avec annotations visibles
fig = go.Figure()
categories = df_price_annotated['market'].unique()
hp_order = ['Très faible', 'Faible', 'Moyenne', 'Élevée', 'Très élevée']

for category in categories:
    df_cat = df_price_annotated[df_price_annotated['market'] == category]
    fig.add_trace(go.Bar(
        x=df_cat['HP Range'],
        y=df_cat['price'],
        name=category,
        text=df_cat['Nombre de véhicules'],
        textposition='outside',  # texte visible au-dessus des barres
        hovertemplate='<b>%{x}</b><br>Catégorie: ' + category + '<br>Prix moyen: $%{y}<br>Nombre de véhicules: %{text}',
    ))

# Mise en forme
fig.update_layout(
    barmode='group',
    title="Prix moyen (price) par catégorie de voiture et puissance moteur (avec nombre de véhicules)",
    xaxis_title="Type de puissance moteur",
    yaxis_title="Prix moyen des véhicules ($)",
    legend_title="Catégorie de véhicule",
    xaxis={'categoryorder': 'array', 'categoryarray': hp_order}
)

fig.show()



C:\Users\jerem\AppData\Local\Temp\ipykernel_19480\838335851.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\jerem\AppData\Local\Temp\ipykernel_19480\838335851.py:13: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [301]:
# Suppression des doublons et des lignes avec valeurs manquantes
df_clean1 = df.dropna(subset=['make', 'model']).copy()
df_clean1 = df_clean1.drop_duplicates(subset=['make', 'model'])

# Comptage des modèles distincts par marque
model_counts = df_clean1.groupby('make')['model'].nunique().reset_index()
model_counts.columns = ['Marque', 'Nombre de modèles']
model_counts = model_counts.sort_values(by='Nombre de modèles', ascending=False)

# Création du bar chart avec une couleur douce et harmonieuse
fig = px.bar(
    model_counts.head(15),
    x='Marque',
    y='Nombre de modèles',
    text='Nombre de modèles',
    title="Top 15 des marques avec le plus grand nombre de modèles distincts",
    color_discrete_sequence=["#4682B4"]  # bleu acier doux
)

# Mise en forme visuelle
fig.update_layout(
    title=dict(x=0.5, font=dict(size=20)),
    font=dict(size=14),
    xaxis=dict(
        title=dict(text="Marque", font=dict(size=16)),
        tickfont=dict(size=13),
        tickangle=-30
    ),
    yaxis=dict(
        title=dict(text="Nombre de modèles", font=dict(size=16)),
        tickfont=dict(size=13),
        gridcolor='lightgrey'
    ),
    plot_bgcolor='white',
    margin=dict(t=80, b=60, l=60, r=40)
)

fig.update_traces(
    textposition="outside",
    marker=dict(line=dict(width=1, color='black'))
)

fig.show()


In [302]:
# Top 5 marques les plus présentes
top_5_makes = model_counts.head(5)['Marque'].tolist()

# Nettoyage
df_clean2 = df.dropna(subset=['make', 'model', 'market']).copy()
df_clean2 = df_clean2.drop_duplicates(subset=['make', 'model', 'market'])

# Explosion des segments multiples
df_clean2['market'] = df_clean2['market'].str.split(',')
df_exploded2 = df_clean2.explode('market')
df_exploded2 = df_exploded2[df_exploded2['make'].isin(top_5_makes)]

# Calcul du nombre de modèles par marque et catégorie de marché
segment_counts = df_exploded2.groupby(['make', 'market']).size().reset_index(name='Nb modèles')

# Création du graphique
fig = px.bar(
    segment_counts,
    x='make',
    y='Nb modèles',
    color='market',
    text='Nb modèles',
    title="Répartition des segments pour les 5 marques principales",
    color_discrete_sequence=px.colors.qualitative.D3  # Palette plus contrastée
)

fig.update_layout(
    barmode='stack',
    title=dict(x=0.5, font=dict(size=20)),
    font=dict(size=14),
    xaxis=dict(
        title=dict(text="Marque", font=dict(size=16)),
        tickangle=0,  # Écriture horizontale
        tickfont=dict(size=13, family='Arial', color='black'),
    ),
    yaxis=dict(
        title=dict(text="Nombre de modèles", font=dict(size=16)),
        tickfont=dict(size=13),
        gridcolor='lightgrey'
    ),
    legend_title_text='Segment (Market Category)',
    plot_bgcolor='white',
    margin=dict(t=80, b=60, l=60, r=40)
)

# Bordures plus fines et lisibles
fig.update_traces(
    textposition='inside',
    marker=dict(line=dict(width=0.5, color='black'))
)

fig.show()

In [303]:
# Vérification des colonnes disponibles dans le DataFrame
print("Colonnes disponibles :", df.columns)

# Nettoyage
df_clean3 = df.dropna(subset=['make', 'model', 'market', 'popularity']).copy()
df_clean3 = df_clean3.drop_duplicates(subset=['make', 'model', 'market'])

# Explosion des catégories
df_clean3['market'] = df_clean3['market'].str.split(',')
df_exploded3 = df_clean3.explode('market')

# Diversité de segments
segment_diversity = df_exploded3.groupby('make')['market'].nunique().reset_index()
segment_diversity.columns = ['make', 'Diversité Segment']

# Fusion avec popularité
popularity_data = df_exploded3.groupby('make')['popularity'].mean().reset_index()
popularity_data.columns = ['make', 'Popularité Moyenne']

# Dataset final
merged_df = pd.merge(segment_diversity, popularity_data, on='make')

# Graphe en boite
fig = px.box(
    merged_df,
    x='Diversité Segment',
    y='Popularité Moyenne',
    color='Diversité Segment',
    title="Distribution de la popularité moyenne selon la diversité des segments",
    labels={
        'Diversité Segment': "Nombre de segments distincts",
        'Popularité Moyenne': "Popularité moyenne"
    }
)

fig.update_layout(
    title=dict(x=0.5, font=dict(size=20)),
    font=dict(size=14),
    xaxis=dict(
        title=dict(text="Diversité des segments", font=dict(size=16)),  # Corrected here
        tickmode='linear',
        tickfont=dict(size=13)
    ),
    yaxis=dict(
        title=dict(text="Popularité moyenne", font=dict(size=16)),
        tickfont=dict(size=13),
        gridcolor='lightgrey'
    ),
    showlegend=False,
    plot_bgcolor='white',
    margin=dict(t=80, b=40, l=60, r=40)
)

fig.update_traces(marker=dict(line=dict(width=1.5)), selector=dict(type='box'))

fig.show()

Colonnes disponibles : Index(['make', 'model', 'year', 'fuel_type', 'hp', 'cylinders', 'transmission',
       'drive', 'doors', 'market', 'size', 'style', 'highway_mpg', 'city_mpg',
       'popularity', 'price', 'average_mpg', 'hp_range'],
      dtype='object')


In [304]:
# Analyse de la dépréciation moyenne par année

# On garde uniquement les voitures dont l'année et le prix sont valides
df_year_price = df[['year', 'price']].dropna()
df_year_price = df_year_price[df_year_price['year'] > 1990]
df_year_price = df_year_price[df_year_price['price'] > 1000]

# Calcul du prix moyen par année
avg_price_by_year = df_year_price.groupby('year')['price'].agg(['mean', 'count']).reset_index()

# Tracé du graphique
fig = px.line(
    avg_price_by_year,
    x='year',
    y='mean',
    markers=True,
    text=avg_price_by_year['count'],
    labels={'year': 'Année du véhicule', 'mean': 'Prix moyen (USD)'},
    title='Dépréciation moyenne des véhicules par année'
)

fig.update_traces(
    hovertemplate='Année: %{x}<br>Prix moyen: $%{y:.2f}<br>Nb véhicules: %{text}<extra></extra>',
    marker=dict(size=6)
)

fig.update_layout(
    yaxis_tickprefix='$',
    yaxis_tickformat=',.0f',
    xaxis_tickangle=-45,
    height=500,
    margin=dict(t=60, b=100),
    hovermode="closest"
)

fig.show()


In [305]:
# Analyse de la dépréciation par style de véhicule

# Garder uniquement les colonnes nécessaires et nettoyer les données
df_style_year = df[['year', 'price', 'style']].dropna()
df_style_year = df_style_year[df_style_year['year'] > 1990]
df_style_year = df_style_year[df_style_year['price'] > 1000]

# Calcul du prix moyen par style et par année
style_group = df_style_year.groupby(['style', 'year'])['price'].agg(['mean', 'count']).reset_index()

# Tracé
fig = px.line(
    style_group,
    x='year',
    y='mean',
    color='style',
    markers=True,
    hover_data={'count': True},
    labels={
        'year': 'Année du véhicule',
        'mean': 'Prix moyen (USD)',
        'style': 'Style de véhicule',
        'count': 'Nb véhicules'
    },
    title='Dépréciation par style de véhicule'
)

fig.update_traces(
    hovertemplate='Style: %{legendgroup}<br>Année: %{x}<br>Prix moyen: $%{y:.2f}<br>Nb véhicules: %{customdata[0]}<extra></extra>',
    marker=dict(size=5)
)

fig.update_layout(
    yaxis_tickprefix='$',
    yaxis_tickformat=',.0f',
    xaxis_tickangle=-45,
    height=600,
    legend_title_text='Style de véhicule',
    margin=dict(t=60, b=120),
    hovermode="closest"
)

fig.show()


In [306]:
# Analyse de la dépréciation par catégorie de marché

# Certaines voitures ont plusieurs catégories séparées par des virgules → on les éclate
df_category = df[['year', 'price', 'market']].dropna()
df_category = df_category[df_category['year'] > 1990]
df_category = df_category[df_category['price'] > 1000]

# Séparer les catégories multiples et réassembler le DataFrame
df_exploded = df_category.assign(
    market=df_category['market'].str.split(',')
).explode('market')

# Nettoyer les noms de catégories
df_exploded['market'] = df_exploded['market'].str.strip()

# Grouper par catégorie + année
cat_group = df_exploded.groupby(['market', 'year'])['price'].agg(['mean', 'count']).reset_index()

# Tracé
fig = px.line(
    cat_group,
    x='year',
    y='mean',
    color='market',
    markers=True,
    hover_data={'count': True},
    labels={
        'year': 'Année du véhicule',
        'mean': 'Prix moyen (USD)',
        'market': 'Catégorie de marché',
        'count': 'Nb véhicules'
    },
    title='Dépréciation par catégorie de marché'
)

fig.update_traces(
    hovertemplate='Catégorie: %{legendgroup}<br>Année: %{x}<br>Prix moyen: $%{y:.2f}<br>Nb véhicules: %{customdata[0]}<extra></extra>',
    marker=dict(size=5)
)

fig.update_layout(
    yaxis_tickprefix='$',
    yaxis_tickformat=',.0f',
    xaxis_tickangle=-45,
    height=600,
    legend_title_text='Catégorie marketing',
    margin=dict(t=60, b=120),
    hovermode="closest"
)

fig.show()
